In [2]:
import os
import numpy as np
import librosa
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score

In [3]:
# === Folder data ===
buka_path = "data/voice/buka-voice"
tutup_path = "data/voice/tutup-voice"

# === Fungsi ekstraksi fitur dari file suara ===
def extract_features(file_path):
    y, sr = librosa.load(file_path, sr=None)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    mfcc_mean = np.mean(mfcc.T, axis=0)
    return mfcc_mean

# === Persiapan data ===
X, y = [], []

for file in os.listdir(buka_path):
    if file.endswith(".mp3"):
        features = extract_features(os.path.join(buka_path, file))
        X.append(features)
        y.append(0)  # 0 untuk 'buka'

for file in os.listdir(tutup_path):
    if file.endswith(".mp3"):
        features = extract_features(os.path.join(tutup_path, file))
        X.append(features)
        y.append(1)  # 1 untuk 'tutup'

In [4]:
X = np.array(X)
y = np.array(y)

# === Split data untuk training dan testing ===
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# === Normalisasi ===
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# === Model KNN ===
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)

# === Evaluasi ===
y_pred = knn.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

# === Simpan model & scaler jika ingin digunakan ulang ===
import joblib
joblib.dump(knn, "model_knn_voice.pkl")
joblib.dump(scaler, "scaler.pkl")

Accuracy: 0.9512195121951219
              precision    recall  f1-score   support

           0       0.95      0.95      0.95        22
           1       0.95      0.95      0.95        19

    accuracy                           0.95        41
   macro avg       0.95      0.95      0.95        41
weighted avg       0.95      0.95      0.95        41



['scaler.pkl']

In [14]:
import sounddevice as sd
import wavio
import joblib
import librosa
import numpy as np

duration = 3  # durasi rekam dalam detik
samplerate = 44100  # sample rate umum

print("🎙️ Mulai merekam...")
recording = sd.rec(int(duration * samplerate), samplerate=samplerate, channels=1, dtype='float32')
sd.wait()
print("✅ Rekaman selesai!")

# Simpan hasil rekaman
file_path = "recorded.wav"
wavio.write(file_path, recording, samplerate, sampwidth=2)

# Load model dan scaler
model = joblib.load("model_knn_voice.pkl")
scaler = joblib.load("scaler.pkl")

# Ekstraksi fitur dan prediksi
def extract_features(file_path):
    y, sr = librosa.load(file_path, sr=None)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    return np.mean(mfcc.T, axis=0)

features = extract_features(file_path).reshape(1, -1)
features = scaler.transform(features)

pred = model.predict(features)
print("Prediksi:", "BUKA" if pred[0] == 0 else "TUTUP")


🎙️ Mulai merekam...
✅ Rekaman selesai!
Prediksi: TUTUP
